# Part 1

In [1]:
!pip install beautifulsoup4
!pip install lxml
!pip install requests

In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
#with open('C:/Users/handa/OneDrive/Desktop/NWU/Courersa/postcode.html',encoding="utf-8") as fp:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data,'lxml')
# print(soup.prettify())

In [4]:
#content in soup.find_all('div',class_ = "mw-parser-output")

In [5]:
import re
postcode = []
borough = []
neighborhood = []
n = 0
for row in soup.find('table').find_all('tr'):
    for element in row.find_all('td'):
        records = element.p.find('span')

        record  = re.split( r'\((.*)\)',records.text)
        
        postcode.append(element.p.b.text)
        borough.append(record[0].replace('/', ','))
        
# if the neighborhood is empty, fill it with Borough
        if len(record)>1:
            neighborhood.append(record[1].replace('/', ','))
        else: 
            neighborhood.append(record[0].replace('/', ','))
    

In [7]:
import pandas as pd
toronto_df = pd.DataFrame({"PostalCode": postcode,
                           "Borough": borough,
                           "neighborhood": neighborhood})

print(toronto_df.shape)
toronto_df.head()


(180, 3)


,PostalCode,Borough,neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"


In [10]:
# Drop the empty Borough rows
toronto_df = toronto_df[toronto_df.Borough != 'Not assigned']
print(df.shape)
toronto_df.head()

(103, 3)


,PostalCode,Borough,neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,"Queen's Park , Ontario Provincial Government","Queen's Park , Ontario Provincial Government"


# Part 2 

In [9]:
# Load the location csv file
url='http://cocl.us/Geospatial_data'
df_pcodes=pd.read_csv(url)
df_pcodes.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
# rename the Postal Code to PostalCode
df_pcodes.columns = ['PostalCode', 'Latitude', 'Longitude']
df_pcodes.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
# Join the data frame according to PostalCode
df=pd.merge(toronto_df,df_pcodes, how='right', on = 'PostalCode')
df.head()

,PostalCode,Borough,neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,"Queen's Park , Ontario Provincial Government","Queen's Park , Ontario Provincial Government",43.662301,-79.389494
